### Monthly provisional counts of deaths by age group and HHS region for select causes of death

In [1]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

In [2]:
file= pd.read_csv("death_by_disease.csv")

In [3]:
death_cause= pd.DataFrame(file)
death_cause

,Date Of Death Year,Date Of Death Month,HHSRegion,AgeGroup,AllCause,NaturalCause,Septicemia (A40-A41),Malignant neoplasms (C00-C97),Diabetes mellitus (E10-E14),Alzheimer disease (G30),...,flag_alz,flag_inflpn,flag_clrd,flag_otherresp,flag_nephr,flag_otherunk,flag_hd,flag_stroke,flag_cov19mcod,flag_cov19ucod
0,2019,1,United States,0-4 years,2022.0,1764.0,17.0,23.0,NaN,0.0,...,NaN,NaN,Suppressed (counts 1-9),NaN,Suppressed (counts 1-9),NaN,NaN,Suppressed (counts 1-9),NaN,NaN
1,2019,1,United States,15-24 years,2340.0,626.0,NaN,123.0,31.0,0.0,...,NaN,NaN,NaN,NaN,Suppressed (counts 1-9),NaN,NaN,NaN,NaN,NaN
2,2019,1,United States,25-34 years,4616.0,1623.0,31.0,268.0,82.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,1,United States,35-44 years,7023.0,4153.0,80.0,955.0,224.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,1,United States,45-54 years,14399.0,11455.0,185.0,3158.0,628.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2965,2021,3,10,5-14 years,14.0,NaN,0.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suppressed (counts 1-9),NaN,NaN
2966,2021,3,10,55-64 years,1182.0,1100.0,NaN,357.0,38.0,NaN,...,Suppressed (counts 1-9),Suppressed (counts 1-9),NaN,NaN,Suppressed (counts 1-9),NaN,NaN,NaN,NaN,NaN
2967,2021,3,10,65-74 years,1984.0,1895.0,NaN,608.0,87.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2968,2021,3,10,75-84 years,2427.0,2333.0,26.0,590.0,89.0,152.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
death_cause.columns

Index(['Date Of Death Year', 'Date Of Death Month', 'HHSRegion', 'AgeGroup',
       'AllCause', 'NaturalCause', 'Septicemia (A40-A41)',
       'Malignant neoplasms (C00-C97)', 'Diabetes mellitus (E10-E14)',
       'Alzheimer disease (G30)', 'Influenza and pneumonia (J09-J18)',
       'Chronic lower respiratory diseases (J40-J47)',
       'Other diseases of respiratory system (J00-J06,J30-J39,J67,J70-J98)',
       'Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)',
       'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)',
       'Diseases of heart (I00-I09,I11,I13,I20-I51)',
       'Cerebrovascular diseases (I60-I69)',
       'COVID-19 (U071, Multiple Cause of Death)',
       'COVID-19 (U071, Underlying Cause of Death)', 'AnalysisDate', 'Note',
       'flag_allcause', 'flag_natcause', 'flag_sept', 'flag_neopl',
       'flag_diab', 'flag_alz', 'flag_inflpn', 'flag_clrd', 'flag_otherresp',
       'flag_nephr', 'flag_ot

In [5]:
death_cause.fillna(0, inplace =True)
death_cause.head()

,Date Of Death Year,Date Of Death Month,HHSRegion,AgeGroup,AllCause,NaturalCause,Septicemia (A40-A41),Malignant neoplasms (C00-C97),Diabetes mellitus (E10-E14),Alzheimer disease (G30),...,flag_alz,flag_inflpn,flag_clrd,flag_otherresp,flag_nephr,flag_otherunk,flag_hd,flag_stroke,flag_cov19mcod,flag_cov19ucod
0,2019,1,United States,0-4 years,2022.0,1764.0,17.0,23.0,0.0,0.0,...,0,0,Suppressed (counts 1-9),0,Suppressed (counts 1-9),0,0,Suppressed (counts 1-9),0,0
1,2019,1,United States,15-24 years,2340.0,626.0,0.0,123.0,31.0,0.0,...,0,0,0,0,Suppressed (counts 1-9),0,0,0,0,0
2,2019,1,United States,25-34 years,4616.0,1623.0,31.0,268.0,82.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2019,1,United States,35-44 years,7023.0,4153.0,80.0,955.0,224.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2019,1,United States,45-54 years,14399.0,11455.0,185.0,3158.0,628.0,13.0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dict = {'Septicemia (A40-A41)': 'Septicemia',
        'Malignant neoplasms (C00-C97)': 'Malignant_neoplasms',
        'Diabetes mellitus (E10-E14)': 'Diabetes_mellitus',
        'Alzheimer disease (G30)': 'Alzheimer_disease',
        'Influenza and pneumonia (J09-J18)': 'Influenza_pneumonia',
        'Chronic lower respiratory diseases (J40-J47)': 'Chronic_lower_respiratory_diseases',
        'Other diseases of respiratory system (J00-J06,J30-J39,J67,J70-J98)': 'Other_diseases_of_respiratory_system',
        'Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)': 'Nephritis',
        'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)':'Symptoms_signs_abnormal_clinical_laboratory_findings_not_elsewhere_classified',
        'Diseases of heart (I00-I09,I11,I13,I20-I51)':'Diseases_of_heart',
        'Cerebrovascular diseases (I60-I69)':'Cerebrovascular_diseases',
        'COVID-19 (U071, Multiple Cause of Death)':'COVID-19_Multiple_Cause_of_Death)',
        'COVID-19 (U071, Underlying Cause of Death)':'COVID-19_Underlying_Cause_of_Death',
        'AnalysisDate':'AnalysisDate'
       }

death_cause.rename(columns=dict,
          inplace=True)

death_cause

,Date Of Death Year,Date Of Death Month,HHSRegion,AgeGroup,AllCause,NaturalCause,Septicemia,Malignant_neoplasms,Diabetes_mellitus,Alzheimer_disease,...,flag_alz,flag_inflpn,flag_clrd,flag_otherresp,flag_nephr,flag_otherunk,flag_hd,flag_stroke,flag_cov19mcod,flag_cov19ucod
0,2019,1,United States,0-4 years,2022.0,1764.0,17.0,23.0,0.0,0.0,...,0,0,Suppressed (counts 1-9),0,Suppressed (counts 1-9),0,0,Suppressed (counts 1-9),0,0
1,2019,1,United States,15-24 years,2340.0,626.0,0.0,123.0,31.0,0.0,...,0,0,0,0,Suppressed (counts 1-9),0,0,0,0,0
2,2019,1,United States,25-34 years,4616.0,1623.0,31.0,268.0,82.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2019,1,United States,35-44 years,7023.0,4153.0,80.0,955.0,224.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2019,1,United States,45-54 years,14399.0,11455.0,185.0,3158.0,628.0,13.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2965,2021,3,10,5-14 years,14.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,Suppressed (counts 1-9),0,0
2966,2021,3,10,55-64 years,1182.0,1100.0,0.0,357.0,38.0,0.0,...,Suppressed (counts 1-9),Suppressed (counts 1-9),0,0,Suppressed (counts 1-9),0,0,0,0,0
2967,2021,3,10,65-74 years,1984.0,1895.0,0.0,608.0,87.0,30.0,...,0,0,0,0,0,0,0,0,0,0
2968,2021,3,10,75-84 years,2427.0,2333.0,26.0,590.0,89.0,152.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
death_cause.columns

Index(['Date Of Death Year', 'Date Of Death Month', 'HHSRegion', 'AgeGroup',
       'AllCause', 'NaturalCause', 'Septicemia', 'Malignant_neoplasms',
       'Diabetes_mellitus', 'Alzheimer_disease', 'Influenza_pneumonia',
       'Chronic_lower_respiratory_diseases',
       'Other_diseases_of_respiratory_system', 'Nephritis',
       'Symptoms_signs_abnormal_clinical_laboratory_findings_not_elsewhere_classified',
       'Diseases_of_heart', 'Cerebrovascular_diseases',
       'COVID-19_Multiple_Cause_of_Death)',
       'COVID-19_Underlying_Cause_of_Death', 'AnalysisDate', 'Note',
       'flag_allcause', 'flag_natcause', 'flag_sept', 'flag_neopl',
       'flag_diab', 'flag_alz', 'flag_inflpn', 'flag_clrd', 'flag_otherresp',
       'flag_nephr', 'flag_otherunk', 'flag_hd', 'flag_stroke',
       'flag_cov19mcod', 'flag_cov19ucod'],
      dtype='object')

In [10]:
no_flags= death_cause.loc[death_cause['Date Of Death Year', 'Date Of Death Month', 'HHSRegion', 'AgeGroup',
       'AllCause', 'NaturalCause', 'Septicemia', 'Malignant_neoplasms',
       'Diabetes_mellitus', 'Alzheimer_disease', 'Influenza_pneumonia',
       'Chronic_lower_respiratory_diseases',
       'Other_diseases_of_respiratory_system', 'Nephritis',
       'Symptoms_signs_abnormal_clinical_laboratory_findings_not_elsewhere_classified',
       'Diseases_of_heart', 'Cerebrovascular_diseases',
       'COVID-19_Multiple_Cause_of_Death)',
       'COVID-19_Underlying_Cause_of_Death', 'AnalysisDate']]

no_flags

KeyError: ('Date Of Death Year', 'Date Of Death Month', 'HHSRegion', 'AgeGroup', 'AllCause', 'NaturalCause', 'Septicemia', 'Malignant_neoplasms', 'Diabetes_mellitus', 'Alzheimer_disease', 'Influenza_pneumonia', 'Chronic_lower_respiratory_diseases', 'Other_diseases_of_respiratory_system', 'Nephritis', 'Symptoms_signs_abnormal_clinical_laboratory_findings_not_elsewhere_classified', 'Diseases_of_heart', 'Cerebrovascular_diseases', 'COVID-19_Multiple_Cause_of_Death)', 'COVID-19_Underlying_Cause_of_Death', 'AnalysisDate')